In [9]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

In [10]:
sysinfo = pd.read_csv('./sysinfo.csv')

In [11]:
sysinfo.head()

,cpu_percent,temperature,timestamp
0,0.38,34,2017-05-24 18:41:53.875155
1,0.86,35,2017-05-24 18:42:53.879360
2,0.12,34,2017-05-24 18:43:53.883331
3,0.97,34,2017-05-24 18:44:53.887442
4,0.38,34,2017-05-24 18:45:53.891643


In [7]:
sysinfo.shape

(921, 3)

In [6]:
sysinfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 921 entries, 0 to 920
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cpu_percent  921 non-null    float64
 1   temperature  921 non-null    int64  
 2   timestamp    921 non-null    object 
dtypes: float64(1), int64(1), object(1)
memory usage: 21.7+ KB


In [12]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sysinfo, y, test_size=0.3, stratify=y, random_state=0)